In [1]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from keras.optimizers import Adam
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar
from matplotlib import pyplot as plt

from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller

from tqdm import tqdm_notebook


from itertools import product
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


Tensorflow/Keras: 2.9.0
pandas: 1.4.3
numpy: 1.23.0
sklearn: 1.1.1
plotly: 5.9.0


In [2]:
df=pd.read_csv('InSAR_data_south/displacement/export_dataframe1.csv')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
df

2014-11-08  2014-12-02  2014-12-26  2015-02-12  \
Longitude Latitude                                                   
-120.9486 34.1347          NaN         NaN         NaN         NaN   
          34.1367          NaN         NaN         NaN         NaN   
          34.1387          NaN         NaN         NaN         NaN   
          34.1407          NaN         NaN         NaN         NaN   
          34.1427          NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
-117.6414 37.6236          0.0      4.8278     -1.3065         NaN   
          37.6256          0.0      4.8698     -1.2137         NaN   
          37.6276          0.0      4.8693     -1.1893         NaN   
          37.6296          0.0      4.8277     -1.1906         NaN   
          37.6316          0.0      4.8729     -1.1200         NaN   

                    2015-03-08  2015-04-01  2015-04-25  2015-05-19  \
Longitude Latitude                                                   
-120.9486 34.1347          NaN         NaN         NaN         NaN   
          34.1367          NaN         NaN         NaN         NaN   
          34.1387          NaN         NaN         NaN         NaN   
          34.1407          NaN         NaN         NaN         NaN   
          34.1427          NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
-117.6414 37.6236      -4.1515     -1.9999      9.9914     -3.1055   
          37.6256      -4.1588     -1.7372     10.4130     -2.9584   
          37.6276      -4.1632     -1.7107     10.4830     -2.9318   
          37.6296      -4.1240     -1.8949     10.1870     -3.0179   
          37.6316      -4.1457     -1.7033     10.5180     -2.9269   

                    2015-06-12  2015-07-06  ...  2018-10-18  2018-10-30  \
Longitude Latitude                          ...                           
-120.9486 34.1347          NaN         NaN  ...         NaN         NaN   
          34.1367          NaN         NaN  ...         NaN         NaN   
          34.1387          NaN         NaN  ...         NaN         NaN   
          34.1407          NaN         NaN  ...         NaN         NaN   
          34.1427          NaN         NaN  ...         NaN         NaN   
...                        ...         ...  ...         ...         ...   
-117.6414 37.6236     -0.32799      6.9899  ...      8.5800      11.341   
          37.6256     -0.33838      7.3402  ...      9.2515      11.996   
          37.6276     -0.34454      7.3978  ...      8.8321      11.570   
          37.6296     -0.31614      7.1245  ...     10.3320      13.073   
          37.6316     -0.33153      7.3839  ...      9.3376      12.053   

                    2018-11-11  2018-11-17  2018-11-23  2018-12-05  \
Longitude Latitude                                                   
-120.9486 34.1347          NaN         NaN         NaN         NaN   
          34.1367          NaN         NaN         NaN         NaN   
          34.1387          NaN         NaN         NaN         NaN   
          34.1407          NaN         NaN         NaN         NaN   
          34.1427          NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
-117.6414 37.6236       5.5704      7.6113      17.002      9.6014   
          37.6256       6.0621      8.2387      17.526      9.5942   
          37.6276       5.5897      7.8226      17.066      9.2927   
          37.6296       7.2137      9.3699      18.582     10.8610   
          37.6316       6.0863      8.3323      17.499      9.8190   

                    2018-12-17  2018-12-29  2019-01-10  2019-01-22  
Longitude Latitude                                                  
-120.9486 34.1347          NaN         NaN         NaN         NaN  
          34.1367          NaN         NaN         NaN         NaN  
          34.1387          NaN     

In [3]:
df=df.dropna(axis=0, how='all')#drop full nan rows
df=df.transpose()
df

Longitude  -120.9146                                                   \
Latitude     35.4470    35.4490  35.4510   35.4530  35.4550   35.4570   
2014-11-08   0.00000   0.000000   0.0000   0.00000   0.0000   0.00000   
2014-12-02 -13.51800 -13.641000 -13.6300 -13.56900 -13.5060 -13.53000   
2014-12-26   0.74746   0.063185   0.4006  -0.50024  -1.1073  -0.80983   
2015-02-12   5.08620   4.415000   4.7172   4.06450   2.9022   3.31120   
2015-03-08  -1.96680  -2.580300  -2.3308  -2.89420  -4.1777  -3.82270   
...              ...        ...      ...       ...      ...       ...   
2018-12-05  -9.62350 -10.564000 -10.1480  -9.09440 -11.3610 -10.89200   
2018-12-17  -4.92490  -5.742900  -5.1576  -3.90210  -6.0692  -5.49590   
2018-12-29  -3.59760  -4.320100  -3.7890  -2.86370  -4.9656  -4.64640   
2019-01-10  -5.13460  -5.820500  -5.1048  -3.86390  -6.0258  -5.53960   
2019-01-22  -4.41930  -5.014400  -4.3554  -3.10450  -5.1433  -4.54840   

Longitude           -120.9126                      ... -117.6414           \
Latitude    35.4590   35.4470   35.4490   35.4510  ...   37.6136  37.6156   
2014-11-08   0.0000    0.0000   0.00000   0.00000  ...   0.00000   0.0000   
2014-12-02 -13.6200  -13.2210 -13.50200 -13.65600  ...   5.00100   5.0127   
2014-12-26  -2.0378    0.4500   0.34938  -0.15311  ...  -0.98686  -1.0013   
2015-02-12   2.3739    4.9826   4.66420   4.02680  ...       NaN      NaN   
2015-03-08  -4.7624   -2.1172  -2.32270  -3.19980  ...  -4.23010  -4.2259   
...             ...       ...       ...       ...  ...       ...      ...   
2018-12-05 -13.6280   -9.6853 -10.52500 -11.33100  ...   9.87220  10.5640   
2018-12-17  -8.1336   -5.1281  -5.84380  -6.37050  ...  17.24800  18.1760   
2018-12-29  -7.3886   -3.6989  -4.46620  -4.88590  ...   9.08440  10.0050   
2019-01-10  -8.2089   -5.3352  -5.92440  -6.19170  ...  12.62500  13.4180   
2019-01-22  -7.0159   -4.7444  -5.35070  -5.47470  ...   8.81900   9.6075   

Longitude                                                                  \
Latitude    37.6176  37.6196  37.6216  37.6236  37.6256  37.6276  37.6296   
2014-11-08   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   
2014-12-02   4.9280   4.9084   4.8957   4.8278   4.8698   4.8693   4.8277   
2014-12-26  -1.1578  -1.1962  -1.2346  -1.3065  -1.2137  -1.1893  -1.1906   
2015-02-12      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
2015-03-08  -4.1743  -4.1856  -4.1966  -4.1515  -4.1588  -4.1632  -4.1240   
...             ...      ...      ...      ...      ...      ...      ...   
2018-12-05  10.3630  10.2660   9.2986   9.6014   9.5942   9.2927  10.8610   
2018-12-17  17.1730  17.4670  16.5890  16.4360  16.7240  16.6410  17.9170   
2018-12-29   9.1781   9.2642   8.0941   8.5234   8.2348   8.2757   9.8912   
2019-01-10  12.7030  12.8120  11.5530  11.9040  11.6300  11.7900  13.3810   
2019-01-22   9.2389   9.3625   8.1547   8.8353   8.3278   8.3947  10.2190   

Longitude            
Latitude    37.6316  
2014-11-08   0.0000  
2014-12-02   4.8729  
2014-12-26  -1.1200  
2015-02-12      NaN  
2015-03-08  -4.1457  
...             ...  
2018-12-05   9.8190  
2018-12-17  16.8390  
2018-12-29   8.8137  
2019-01-10  12.2150  
2019-01-22   8.8725  

[110 rows x 1657958 columns]

In [4]:
df.index

DatetimeIndex(['2014-11-08', '2014-12-02', '2014-12-26', '2015-02-12',
               '2015-03-08', '2015-04-01', '2015-04-25', '2015-05-19',
               '2015-06-12', '2015-07-06',
               ...
               '2018-10-18', '2018-10-30', '2018-11-11', '2018-11-17',
               '2018-11-23', '2018-12-05', '2018-12-17', '2018-12-29',
               '2019-01-10', '2019-01-22'],
              dtype='datetime64[ns]', length=110, freq=None)

In [5]:
#data = df.set_axis(['data'], axis=1, inplace=False)
#data.insert(0,"date",value=data.index)
data=df.reset_index()
data


Longitude      index -120.9146                                         \
Latitude                35.447     35.449   35.451    35.453   35.455   
0         2014-11-08   0.00000   0.000000   0.0000   0.00000   0.0000   
1         2014-12-02 -13.51800 -13.641000 -13.6300 -13.56900 -13.5060   
2         2014-12-26   0.74746   0.063185   0.4006  -0.50024  -1.1073   
3         2015-02-12   5.08620   4.415000   4.7172   4.06450   2.9022   
4         2015-03-08  -1.96680  -2.580300  -2.3308  -2.89420  -4.1777   
..               ...       ...        ...      ...       ...      ...   
105       2018-12-05  -9.62350 -10.564000 -10.1480  -9.09440 -11.3610   
106       2018-12-17  -4.92490  -5.742900  -5.1576  -3.90210  -6.0692   
107       2018-12-29  -3.59760  -4.320100  -3.7890  -2.86370  -4.9656   
108       2019-01-10  -5.13460  -5.820500  -5.1048  -3.86390  -6.0258   
109       2019-01-22  -4.41930  -5.014400  -4.3554  -3.10450  -5.1433   

Longitude                    -120.9126            ... -117.6414           \
Latitude     35.457   35.459    35.447    35.449  ...   37.6136  37.6156   
0           0.00000   0.0000    0.0000   0.00000  ...   0.00000   0.0000   
1         -13.53000 -13.6200  -13.2210 -13.50200  ...   5.00100   5.0127   
2          -0.80983  -2.0378    0.4500   0.34938  ...  -0.98686  -1.0013   
3           3.31120   2.3739    4.9826   4.66420  ...       NaN      NaN   
4          -3.82270  -4.7624   -2.1172  -2.32270  ...  -4.23010  -4.2259   
..              ...      ...       ...       ...  ...       ...      ...   
105       -10.89200 -13.6280   -9.6853 -10.52500  ...   9.87220  10.5640   
106        -5.49590  -8.1336   -5.1281  -5.84380  ...  17.24800  18.1760   
107        -4.64640  -7.3886   -3.6989  -4.46620  ...   9.08440  10.0050   
108        -5.53960  -8.2089   -5.3352  -5.92440  ...  12.62500  13.4180   
109        -4.54840  -7.0159   -4.7444  -5.35070  ...   8.81900   9.6075   

Longitude                                                                 \
Latitude   37.6176  37.6196  37.6216  37.6236  37.6256  37.6276  37.6296   
0           0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   0.0000   
1           4.9280   4.9084   4.8957   4.8278   4.8698   4.8693   4.8277   
2          -1.1578  -1.1962  -1.2346  -1.3065  -1.2137  -1.1893  -1.1906   
3              NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4          -4.1743  -4.1856  -4.1966  -4.1515  -4.1588  -4.1632  -4.1240   
..             ...      ...      ...      ...      ...      ...      ...   
105        10.3630  10.2660   9.2986   9.6014   9.5942   9.2927  10.8610   
106        17.1730  17.4670  16.5890  16.4360  16.7240  16.6410  17.9170   
107         9.1781   9.2642   8.0941   8.5234   8.2348   8.2757   9.8912   
108        12.7030  12.8120  11.5530  11.9040  11.6300  11.7900  13.3810   
109         9.2389   9.3625   8.1547   8.8353   8.3278   8.3947  10.2190   

Longitude           
Latitude   37.6316  
0           0.0000  
1           4.8729  
2          -1.1200  
3              NaN  
4          -4.1457  
..             ...  
105         9.8190  
106        16.8390  
107         8.8137  
108        12.2150  
109         8.8725  

[110 rows x 1657959 columns]

In [20]:
data3=data.resample("Y", on="index").agg('mean').reset_index(drop=False)

In [21]:
data3

Longitude      index -120.9146                                          \
Latitude                35.447    35.449    35.451    35.453    35.455   
0         2014-12-31 -4.256847 -4.525938 -4.409800 -4.689747 -4.871100   
1         2015-12-31 -2.880327 -3.453890 -3.211751 -3.679129 -5.010846   
2         2016-12-31 -2.985137 -3.266293 -2.738777 -3.154578 -4.271317   
3         2017-12-31 -2.926110 -3.090085 -2.391249 -2.033723 -3.789125   
4         2018-12-31 -4.371226 -4.981626 -4.416182 -3.465114 -5.629486   
5         2019-12-31 -4.776950 -5.417450 -4.730100 -3.484200 -5.584550   

Longitude                     -120.9126            ...  -117.6414             \
Latitude     35.457    35.459    35.447    35.449  ...    37.6136    37.6156   
0         -4.779943 -5.219267 -4.257000 -4.384207  ...   1.338047   1.337133   
1         -4.503429 -5.759162 -3.052912 -3.155797  ...   2.356124   2.410873   
2         -3.796648 -5.303146 -3.121606 -3.350303  ...   3.576591   3.648314   
3         -2.653271 -5.057682 -2.895597 -3.349556  ...  10.318065  10.555761   
4         -4.945945 -7.650468 -4.332731 -5.008810  ...  10.750750  11.074878   
5         -5.044000 -7.612400 -5.039800 -5.637550  ...  10.722000  11.512750   

Longitude                                                                   \
Latitude     37.6176    37.6196   37.6216    37.6236    37.6256    37.6276   
0           1.256733   1.237400  1.220367   1.173767   1.218700   1.226667   
1           2.175242   2.132741  2.145209   1.976954   2.119207   2.143026   
2           3.507111   3.466514  3.484604   3.392095   3.496180   3.551212   
3           9.812568  10.323497  9.561355   9.259861   9.868855   9.472877   
4          10.314481  10.824325  9.969016   9.657928  10.296912   9.904988   
5          10.970950  11.087250  9.853850  10.369650   9.978900  10.092350   

Longitude                        
Latitude     37.6296    37.6316  
0           1.212367   1.250967  
1           2.039883   2.124737  
2           3.506055   3.586725  
3          10.871381   9.980961  
4          11.386234  10.410356  
5          11.800000  10.543750  

[6 rows x 1657959 columns]

In [22]:
data3=data3.transpose()


In [24]:
data3.columns=data3.iloc[0].values


In [25]:
data3=data3[1:]

In [26]:
data3.index

MultiIndex([(-120.9146,  35.447),
            (-120.9146,  35.449),
            (-120.9146,  35.451),
            (-120.9146,  35.453),
            (-120.9146,  35.455),
            (-120.9146,  35.457),
            (-120.9146,  35.459),
            (-120.9126,  35.447),
            (-120.9126,  35.449),
            (-120.9126,  35.451),
            ...
            (-117.6414, 37.6136),
            (-117.6414, 37.6156),
            (-117.6414, 37.6176),
            (-117.6414, 37.6196),
            (-117.6414, 37.6216),
            (-117.6414, 37.6236),
            (-117.6414, 37.6256),
            (-117.6414, 37.6276),
            (-117.6414, 37.6296),
            (-117.6414, 37.6316)],
           names=['Longitude', 'Latitude'], length=1657958)

In [27]:
data3

2014-12-31 2015-12-31 2016-12-31 2017-12-31 2018-12-31  \
Longitude Latitude                                                          
-120.9146 35.4470   -4.256847  -2.880327  -2.985137   -2.92611  -4.371226   
          35.4490   -4.525938   -3.45389  -3.266293  -3.090085  -4.981626   
          35.4510     -4.4098  -3.211751  -2.738777  -2.391249  -4.416182   
          35.4530   -4.689747  -3.679129  -3.154578  -2.033723  -3.465114   
          35.4550     -4.8711  -5.010846  -4.271317  -3.789125  -5.629486   
...                       ...        ...        ...        ...        ...   
-117.6414 37.6236    1.173767   1.976954   3.392095   9.259861   9.657928   
          37.6256      1.2187   2.119207    3.49618   9.868855  10.296912   
          37.6276    1.226667   2.143026   3.551212   9.472877   9.904988   
          37.6296    1.212367   2.039883   3.506055  10.871381  11.386234   
          37.6316    1.250967   2.124737   3.586725   9.980961  10.410356   

                   2019-12-31  
Longitude Latitude             
-120.9146 35.4470    -4.77695  
          35.4490    -5.41745  
          35.4510     -4.7301  
          35.4530     -3.4842  
          35.4550    -5.58455  
...                       ...  
-117.6414 37.6236    10.36965  
          37.6256      9.9789  
          37.6276    10.09235  
          37.6296        11.8  
          37.6316    10.54375  

[1657958 rows x 6 columns]

In [9]:
def shaping(datain, timestep):
    
    # Convert input dataframe to array and flatten
    arr=datain.to_numpy().flatten() 
    arr[np.isnan(arr)] = -100
    #print(arr)
    cnt=0
    for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range lenght of the dates - 2* amount of timesep?? +1
        cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
        X_start=mth # Start month for inputs of each sample
        X_end=mth+timestep # End month for inputs of each sample
        Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
        Y_end=mth+2*timestep # End month for targets of each sample.  
        
        # Assemble input and target arrays containing all samples
        if mth==0:
            X_comb=arr[X_start:X_end]
            Y_comb=arr[Y_start:Y_end]
        else: 
            X_comb=np.append(X_comb, arr[X_start:X_end])
            Y_comb=np.append(Y_comb, arr[Y_start:Y_end])
    
    # Reshape input and target arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [10]:
##### Step 2 - Prepare data
tup=(-119.9588,35.185)
#tup=(-119.2990,35.8852)
#tup=(-120.2888,34.8349)
#tup=(-117.6414,37.6236)
# Split data into train and test dataframes
df_train_temp=df.iloc[:, :88].copy()
df_test_temp=df.iloc[:, 88:110].copy()
#pick a location to test
df_train = df_train_temp[df_train_temp.index==tup].copy()
df_test = df_test_temp[df_test_temp.index==tup].copy()

print(df_test.index)

ValueError: Lengths must match

In [ ]:
tup=(-119.9588,35.185)

data=df[df.index==tup].copy()
print(data.columns[0])
print(data.iloc[0].values.size)
#data=df[df.index==tup].copy()
#data=df.loc(tup)
#data


In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(data.columns,data.iloc[0].values)
plt.title('-119.9588,35.185')
plt.ylabel('Displacement')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)

for year in range(2014,2019):
    plt.axvline(pd.to_datetime(str(year)+"-01-01"), color='k', linestyle='--', alpha=0.2)
plt.show()

In [ ]:
data=data.drop(columns=data.columns[0:3],axis=1)

data=data.drop(columns=data.columns[105:108],axis=1)
data

In [ ]:
data=data.transpose()


In [ ]:

data = data.set_axis(['data'], axis=1, inplace=False)
#data.insert(0,"date",value=data.index)
data=data.reset_index()
data
print(data)

In [ ]:
data2=data.resample("M", on="index").first().dropna().reset_index(drop=True)

In [ ]:

#data=data.transpose()
data2

#data2 = data.set_axis(['data'], axis=1, inplace=False)
#print(data2)

In [ ]:
plot_pacf(data2.data.dropna(), lags=20);
plot_acf(data2.data.dropna(), lags=20);

In [ ]:
data3=data
data3


In [ ]:
data3=data3.resample("M", on="index").agg('mean').dropna().reset_index(drop=False)
print(data3)


In [ ]:
plot_pacf(data3.data.dropna(), lags=20);
plot_acf(data3.data.dropna(), lags=20);

In [ ]:

data3.rename(columns={'index':'date'}, inplace=True)
data3

In [ ]:
plt.figure(figsize=[15, 7.5]); # Set dimensions for figure
plt.plot(data3.date.values,data3.data.values)
plt.title('-119.9588,35.185')
plt.ylabel('Displacement')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.grid(True)


plt.show()